In [ ]:
import os, pandas as pd, numpy as np
#stolen from https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533 
train_names, train_texts = [], []
for f in list(os.listdir('../input/feedback-prize-2021/train')):
    train_names.append(f.replace('.txt', ''))
    train_texts.append(open('../input/feedback-prize-2021/train/' + f, 'r').read())
train_texts = pd.DataFrame({'id': train_names, 'text': train_texts})
train_texts.head()
train = pd.read_csv("../input/feedback-prize-2021/train.csv")

In [ ]:
print(train.keys())
df = pd.pivot_table(train, index=['id'],columns=['discourse_type'], aggfunc='count')['discourse_type_num']
print(df.keys())
df[(df["Lead"] > 0) & (np.isnan(df["Position"]))]

In [ ]:
from tqdm import tqdm 
retval = []
c = 0
for tti in tqdm(range(len(train_texts))):
    row = train_texts.iloc[tti]
    idv = row['id']
    full_text = row['text']
    q = train.query(f"id == '{idv}'")
    for i,r in q.iterrows(): 
        c = c + 1
        discourse_text = r['discourse_text']
        prs = r['predictionstring']
        char_start = int(r['discourse_start'])
        char_end = int(r['discourse_end'])
        word_start = len(full_text[:char_start].split())
        word_end = word_start + len(full_text[char_start:char_end].split())
        if word_end  > len(full_text.split()):
            word_end = word_end - 1
        predictionstring = " ".join( [str(x) for x in range(word_start,word_end)] )
        if prs != predictionstring:
            retval.append((idv, r['discourse_id'], prs, predictionstring))
print(c)

In [ ]:
print(f"Train Length: {np.shape(train)}, c: {c} retval: {len(retval)} retval/train: {len(retval)/len(train)}")

In [ ]:
df = pd.DataFrame(retval, columns = ['idv', 'di', 'prs', 'preds'])
dis = df['idv']
nw = train_texts[np.isin(train_texts['id'], dis)]
nretval = []
for i,r in nw.iterrows():
    tx = r['text']
    if "Generic_Name" in tx:
        print("Generic_Name")
    elif "Generic_School" in tx:
        print("Generic_Name")
    elif "PROPER_NAME" in tx:
        print("PROPER_NAME")
    elif "SCHOOL_NAME" in tx:
        print("SCHOOL_NAME")
    else:
        nretval.append(r['id'])
print(len(nretval))

In [ ]:
def calc_word_indices(full_text, discourse_start, discourse_end):
    start_index = len(full_text[:discourse_start].split())
    token_len = len(full_text[discourse_start:discourse_end].split())
    end = start_index + token_len
    if end > len(full_text.split()):
        end = end - 1
    return list(range(start_index, end))


from tqdm import tqdm 
retval = []
c = 0
for tti in tqdm(range(len(train_texts))):
    row = train_texts.iloc[tti]
    idv = row['id']
    full_text = row['text']
    q = train.query(f"id == '{idv}'")
    for i,r in q.iterrows(): 
        c = c + 1
        discourse_text = r['discourse_text']
        prs = r['predictionstring']
        char_start = int(r['discourse_start'])
        char_end = int(r['discourse_end'])
        rng = calc_word_indices(full_text, char_start, char_end)
        predictionstring = " ".join( [str(x) for x in rng] )
        if prs != predictionstring:
            retval.append((idv, r['discourse_id'], prs, predictionstring))
print(c)
len(retval)

In [ ]:
len(retval)